# Langchain examples

## Simple SQL database chain

In [1]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
import yaml
import os

Install Chinook database following:
https://database.guide/2-sample-databases-sqlite/

In [2]:
# Load the config file
with open('../../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Get the value of the environment variable from the config
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [6]:
db = SQLDatabase.from_uri("sqlite:///../../Chinook.db")
llm = OpenAI(temperature=0)

In [7]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [8]:
db_chain.run("how many tables existing in the database. What are their names?")



> Entering new SQLDatabaseChain chain...
how many tables existing in the database. What are their names? 
SQLQuery:

/Users/dmaguire/Library/Caches/pypoetry/virtualenvs/llm-tool-sandbox-O45YYzTu-py3.10/lib/python3.10/site-packages/langchain/sql_database.py:191: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows = connection.execute(command)


 SELECT name FROM sqlite_master WHERE type='table';
SQLResult: [('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
Answer: There are 10 tables in the database: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track.
> Finished chain.


' There are 10 tables in the database: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track.'

In [9]:
db_chain.run("what does the table InvoiceLine look like?")



> Entering new SQLDatabaseChain chain...
what does the table InvoiceLine look like? 
SQLQuery: SELECT * FROM InvoiceLine LIMIT 5;
SQLResult: [(1, 1, 2, 0.99, 1), (2, 1, 4, 0.99, 1), (3, 2, 6, 0.99, 1), (4, 2, 8, 0.99, 1), (5, 2, 10, 0.99, 1)]
Answer: The InvoiceLine table contains 5 columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, and Quantity.
> Finished chain.


' The InvoiceLine table contains 5 columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, and Quantity.'

In [17]:
db_chain.run("Which artist has the most albums?")



> Entering new SQLDatabaseChain chain...
Which artist has the most albums? 
SQLQuery: SELECT Name, COUNT(*) AS AlbumCount FROM Artist JOIN Album ON Artist.ArtistId = Album.ArtistId GROUP BY Name ORDER BY AlbumCount DESC LIMIT 5;
SQLResult: [('Iron Maiden', 21), ('Led Zeppelin', 14), ('Deep Purple', 11), ('U2', 10), ('Metallica', 10)]
Answer: Iron Maiden has the most albums with 21.
> Finished chain.


' Iron Maiden has the most albums with 21.'

In [18]:
db_chain.run("Which artist has the most appearances on playsts?")



> Entering new SQLDatabaseChain chain...
Which artist has the most appearances on playsts? 
SQLQuery: SELECT Artist.Name, COUNT(PlaylistTrack.TrackId) AS Appearances 
FROM Artist 
INNER JOIN Album ON Artist.ArtistId = Album.ArtistId 
INNER JOIN Track ON Album.AlbumId = Track.AlbumId 
INNER JOIN PlaylistTrack ON Track.TrackId = PlaylistTrack.TrackId 
GROUP BY Artist.Name 
ORDER BY Appearances DESC 
LIMIT 5;
SQLResult: [('Iron Maiden', 516), ('U2', 333), ('Metallica', 296), ('Led Zeppelin', 252), ('Deep Purple', 226)]
Answer: Iron Maiden has the most appearances on playlists with 516.
> Finished chain.


' Iron Maiden has the most appearances on playlists with 516.'

In [10]:
result = db_chain.run("Which artist has the most appearances on playsts?")



> Entering new SQLDatabaseChain chain...
Which artist has the most appearances on playsts? 
SQLQuery: SELECT Artist.Name, COUNT(PlaylistTrack.TrackId) AS Appearances 
FROM Artist 
INNER JOIN Album ON Artist.ArtistId = Album.ArtistId 
INNER JOIN Track ON Album.AlbumId = Track.AlbumId 
INNER JOIN PlaylistTrack ON Track.TrackId = PlaylistTrack.TrackId 
GROUP BY Artist.Name 
ORDER BY Appearances DESC 
LIMIT 5;
SQLResult: [('Iron Maiden', 516), ('U2', 333), ('Metallica', 296), ('Led Zeppelin', 252), ('Deep Purple', 226)]
Answer: Iron Maiden has the most appearances on playlists with 516.
> Finished chain.


### Customize Prompt

In [13]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone asks for the table foobar, they really mean the employee table.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=PROMPT, verbose=True)
db_chain.run("How many employees are there in the foobar table?")



> Entering new SQLDatabaseChain chain...
How many employees are there in the foobar table? 
SQLQuery: SELECT COUNT(*) FROM Employee;
SQLResult: [(8,)]
Answer: There are 8 employees in the foobar table.
> Finished chain.


' There are 8 employees in the foobar table.'

### Return Intermediate Steps

In [16]:
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=PROMPT, verbose=True, return_intermediate_steps=True)

In [17]:
result = db_chain("How many employees are there in the foobar table?")
result["intermediate_steps"]



> Entering new SQLDatabaseChain chain...
How many employees are there in the foobar table? 
SQLQuery: SELECT COUNT(*) FROM Employee;
SQLResult: [(8,)]
Answer: There are 8 employees in the foobar table.
> Finished chain.


[' SELECT COUNT(*) FROM Employee;', '[(8,)]']

### Limit Rows Returned

In [18]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, top_k=3)
db_chain.run("What are some example tracks by composer Johann Sebastian Bach?")



> Entering new SQLDatabaseChain chain...
What are some example tracks by composer Johann Sebastian Bach? 
SQLQuery: SELECT Name, Composer FROM Track WHERE Composer LIKE '%Johann Sebastian Bach%' LIMIT 3;
SQLResult: [('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Johann Sebastian Bach'), ('Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', 'Johann Sebastian Bach'), ('Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude', 'Johann Sebastian Bach')]
Answer: Some example tracks by composer Johann Sebastian Bach are 'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', and 'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude'.
> Finished chain.


' Some example tracks by composer Johann Sebastian Bach are \'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace\', \'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria\', and \'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude\'.'

### Adding example rows from each table

In [20]:
db = SQLDatabase.from_uri(
    "sqlite:///../../Chinook.db",
    include_tables=['Track'], # we include only one table to save tokens in the prompt :)
    sample_rows_in_table_info=2)

In [21]:
print(db.table_info)


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
2 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	1	1	Angus Young, Malcolm Young, Brian Johnson	343719	11170334	0.99
2	Balls to the Wall	2	2	1	None	342562	5510424	0.99
*/


/Users/dmaguire/Library/Caches/pypoetry/virtualenvs/llm-tool-sandbox-O45YYzTu-py3.10/lib/python3.10/site-packages/langchain/sql_database.py:191: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows = connection.execute(command)


In [22]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [23]:
db_chain.run("What are some example tracks by Bach?")



> Entering new SQLDatabaseChain chain...
What are some example tracks by Bach? 
SQLQuery: SELECT TrackId, Name, Composer FROM Track WHERE Composer LIKE '%Bach%' LIMIT 5;
SQLResult: [(1709, 'American Woman', 'B. Cummings/G. Peterson/M.J. Kale/R. Bachman'), (3407, 'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Johann Sebastian Bach'), (3408, 'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', 'Johann Sebastian Bach'), (3409, 'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude', 'Johann Sebastian Bach'), (3430, 'Toccata and Fugue in D Minor, BWV 565: I. Toccata', 'Johann Sebastian Bach')]
Answer: Some example tracks by Bach are American Woman, Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace, Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria, Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude, and Toccata and Fugue in D Minor, BWV 565: I. Toccata.
> Finished chain.


' Some example tracks by Bach are American Woman, Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace, Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria, Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude, and Toccata and Fugue in D Minor, BWV 565: I. Toccata.'

In [25]:
custom_table_info = {
    "Track": """CREATE TABLE Track (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL,
	"Composer" NVARCHAR(220),
	PRIMARY KEY ("TrackId")
)
/*
3 rows from Track table:
TrackId	Name	Composer
1	For Those About To Rock (We Salute You)	Angus Young, Malcolm Young, Brian Johnson
2	Balls to the Wall	None
3	My favorite song ever	The coolest composer of all time
*/"""
}
db = SQLDatabase.from_uri(
    "sqlite:///../../Chinook.db",
    include_tables=['Track', 'Playlist'],
    sample_rows_in_table_info=2,
    custom_table_info=custom_table_info)

print(db.table_info)


CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
2 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
*/

CREATE TABLE Track (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL,
	"Composer" NVARCHAR(220),
	PRIMARY KEY ("TrackId")
)
/*
3 rows from Track table:
TrackId	Name	Composer
1	For Those About To Rock (We Salute You)	Angus Young, Malcolm Young, Brian Johnson
2	Balls to the Wall	None
3	My favorite song ever	The coolest composer of all time
*/


In [26]:
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
db_chain.run("What are some example tracks by Bach?")



> Entering new SQLDatabaseChain chain...
What are some example tracks by Bach? 
SQLQuery: SELECT Name, Composer FROM Track WHERE Composer LIKE '%Bach%' LIMIT 5;
SQLResult: [('American Woman', 'B. Cummings/G. Peterson/M.J. Kale/R. Bachman'), ('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Johann Sebastian Bach'), ('Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', 'Johann Sebastian Bach'), ('Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude', 'Johann Sebastian Bach'), ('Toccata and Fugue in D Minor, BWV 565: I. Toccata', 'Johann Sebastian Bach')]
Answer: Some example tracks by Bach are 'American Woman', 'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', 'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude', and 'Toccata and Fugue in D Minor, BWV 565: I. Toccata'.
> Finished chain.


' Some example tracks by Bach are \'American Woman\', \'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace\', \'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria\', \'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude\', and \'Toccata and Fugue in D Minor, BWV 565: I. Toccata\'.'

## Sequential Chain

1. Based on the query, determine which tables to use.
2. Based on those tables, call the normal SQL database chain.

This is useful for databases with many tables.

In [27]:
from langchain.chains import SQLDatabaseSequentialChain
db = SQLDatabase.from_uri("sqlite:///../../Chinook.db")

In [28]:
chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

In [29]:
chain.run("How many employees are also customers?")



> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['Customer', 'Employee']

> Entering new SQLDatabaseChain chain...
How many employees are also customers? 
SQLQuery: SELECT COUNT(*) FROM Employee e INNER JOIN Customer c ON e.EmployeeId = c.SupportRepId;
SQLResult: [(59,)]
Answer: 59 employees are also customers.
> Finished chain.

> Finished chain.


' 59 employees are also customers.'